In [33]:
import numpy as np
from sklearn.datasets import make_spd_matrix
import cvxpy as cp
import time
import pandas as pd
import matplotlib.pyplot as plt

In [34]:
np.random.seed(42)

In [35]:
def quadform_grad(A, x, mu):  # Complexity of 1st oracle is n + n^2
    return A @ (x - mu) * 2

In [36]:
def atom_quadform_grad(A, x, mu, i):
    return np.dot(A[:, i], x - mu) * 2

In [37]:
def constraint_grad(x):
    return 2 * x

In [38]:
def project_to_constraint(x):
    x_norm = np.linalg.norm(x)
    return x if x_norm <= 1 else x / x_norm

In [39]:
def is_kkt_optimal(grad_obj, grad_constraint, eps=0.01):

    dot = np.dot(-grad_obj, grad_constraint)
    norm_f = np.linalg.norm(grad_obj)
    norm_g = np.linalg.norm(grad_constraint)
    if norm_f < eps or norm_g < eps:
        return True
    cos_angle = dot / (norm_f * norm_g)

    return bool((np.abs(cos_angle - 1) < eps).all())

In [40]:
def solve_elipsoid(A, mu):

    x = cp.Variable(A.shape[0])

    objective = cp.Minimize(cp.quad_form(x - mu, A))

    constraints = [cp.norm2(x) <= 1]

    prob = cp.Problem(objective, constraints)

    result = prob.solve()

    return x.value, result

In [41]:
def str_cvx_gd_solve_elipsoid(
    A, mu, x_init, f_star, eps=0.01, max_iters=10000, return_history=False
):

    alpha = np.min(np.linalg.eigvals(A * 2))

    lr = 2 / alpha

    x = x_init.copy()

    iterations = 0
    operations = 0

    precision_history = [(x - mu).T @ A @ (x - mu) - f_star]

    while iterations < max_iters:

        grad = quadform_grad(A, x, mu)
        x_new = project_to_constraint(x - lr / (iterations + 1) * grad)
        f_val = (x_new - mu).T @ A @ (x_new - mu)

        x = x_new
        iterations += 1

        precision_history.append(abs(f_val - f_star))

        if abs(f_val - f_star) <= eps:
            break

    if return_history:
        return x_new, iterations, f_val, precision_history

    return x_new, f_val, iterations, operations

In [42]:
def str_cvx_sgd_solve_elipsoid(
    A,
    mu,
    x_init,
    f_star,
    dim_sample_ratio="single", # MAY BE FLOAT in (0, 1]
    eps=0.01,
    max_iters=10000,
    return_history=False,
):

    n_dims = len(mu)

    if dim_sample_ratio == "single":
        m_dims = 1

    else:
        m_dims = int(dim_sample_ratio * n_dims)

    dim_sample_ratio = m_dims/n_dims

    alpha = np.min(np.linalg.eigvals(A * 2))

    lr = 2 / (alpha*dim_sample_ratio)

    x_new = x_init.copy()

    iterations = 0
    operations = 0

    precision_history = [(x_new - mu).T @ A @ (x_new - mu) - f_star]

    while iterations < max_iters:

        step_lr = lr / (iterations + 1)

        sampled_dims = np.random.choice(n_dims, size=m_dims, replace=False)

        for i in sampled_dims:
            x_new[i] -= step_lr * atom_quadform_grad(A, x_new, mu, i)

        x_new = project_to_constraint(x_new)

        f_val = (x_new - mu).T @ A @ (x_new - mu)

        x = x_new
        iterations += 1

        precision_history.append(abs(f_val - f_star))

        if abs(f_val - f_star) <= eps:
            break

    if return_history:
        return x_new, iterations, f_val, precision_history

    return x_new, f_val, iterations, operations

In [43]:
def rand_elipsoids_sol_stats_gd(
    n_dim: int = 2, n_samples: int = 100, n_init_states = 100, random_states: list = None, eps=0.01
):
    mu = np.ones(n_dim)

    if random_states is None:
        random_states = np.random.randint(0, 4294967295, size=n_samples)

    affine_ops = [make_spd_matrix(n_dim, random_state=state) for state in random_states]
    
    cases = []

    for i, A in enumerate(affine_ops):

        solution = solve_elipsoid(A, mu)

        eig_vigals = np.linalg.eigvals(A)
        conditional_number = max(eig_vigals) / min(eig_vigals)


        for _ in range(n_init_states):

            x_init = project_to_constraint(np.random.randn(n_dim))
            x_opt, f_val, iters, ops = str_cvx_gd_solve_elipsoid(A, mu, x_init, solution[-1], eps=eps)
            
            cases.append({
                "n_dims": n_dim,
                "example_id" : i,
                "conditional_number": conditional_number,
                "Lipschitz_L" : max(eig_vigals),
                "strong_convexity_alpha": min(eig_vigals)*2,
                "is_kkt_optimal": is_kkt_optimal(quadform_grad(A, x_opt, mu), constraint_grad(x_opt)),
                "solution_delta": np.abs(solution[-1] - f_val),
                "iters" : iters,
                "ops" : ops
            })

    return pd.DataFrame.from_records(cases)

In [ ]:
def rand_elipsoids_sol_stats_sgd(
    n_dim: int = 2, n_samples: int = 100, n_init_states = 100, dim_sample_ratio = "single",random_states: list = None, eps=0.01
):
    mu = np.ones(n_dim)

    if random_states is None:
        random_states = np.random.randint(0, 4294967295, size=n_samples)

    affine_ops = [make_spd_matrix(n_dim, random_state=state) for state in random_states]
    
    cases = []

    if dim_sample_ratio == "single":
        m_dims = 1

    else:
        m_dims = int(dim_sample_ratio * n_dim)

    for i, A in enumerate(affine_ops):

        solution = solve_elipsoid(A, mu)

        eig_vigals = np.linalg.eigvals(A)
        conditional_number = max(eig_vigals) / min(eig_vigals)


        for _ in range(n_init_states):

            x_init = project_to_constraint(np.random.randn(n_dim))
            x_opt, f_val, iters, ops = str_cvx_sgd_solve_elipsoid(A, mu, x_init, solution[-1], dim_sample_ratio, eps=eps)
            

            cases.append({
                "n_dims": n_dim,
                "example_id" : i,
                "conditional_number": conditional_number,
                "Lipschitz_L" : max(eig_vigals),
                "strong_convexity_alpha": min(eig_vigals)*2,
                "is_kkt_optimal": is_kkt_optimal(quadform_grad(A, x_opt, mu), constraint_grad(x_opt)),
                "solution_delta": np.abs(solution[-1] - f_val),
                "iters" : iters,
                "ops" : ops,
                "m_dims": m_dims
            })

    return pd.DataFrame.from_records(cases)

In [51]:
def make_gd_full_stats(n_dims = None):

    if n_dims is None:
        n_dims = range(10, 110, 10)

    df_l = []

    for n in n_dims:
        df_l.append(rand_elipsoids_sol_stats_gd(n))

    return pd.concat(df_l)

In [ ]:
def make_sgd_full_stats(n_dims = None, m_ratios = None):

    if n_dims is None:
        n_dims = range(10, 110, 10)

    if m_ratios is None:
        m_ratios = ["single", 1/8, 1/4, 1/2, 1]

    df_l = []

    for n in n_dims:
        for r in m_ratios:
            df_l.append(rand_elipsoids_sol_stats_sgd(n_dim=n, dim_sample_ratio=r))

    return pd.concat(df_l)

In [53]:
# gd_stats = make_gd_full_stats([2, 4])
sgd_stats = make_sgd_full_stats([16, 32])

In [54]:
# gd_stats

In [55]:
sgd_stats

,n_dims,example_id,conditional_number,Lipschitz_L,strong_convexity_alpha,is_kkt_optimal,solution_delta,iters,ops,m_dims
0,16,0,937.109189,16.397107,0.034995,True,0.009483,9176,0,2
1,16,0,937.109189,16.397107,0.034995,False,0.091247,10000,0,2
2,16,0,937.109189,16.397107,0.034995,True,0.009430,7138,0,2
3,16,0,937.109189,16.397107,0.034995,True,0.007736,8800,0,2
4,16,0,937.109189,16.397107,0.034995,True,0.009232,8506,0,2
...,...,...,...,...,...,...,...,...,...,...
95,32,9,2776.536580,32.453638,0.023377,False,0.105917,10000,0,16
96,32,9,2776.536580,32.453638,0.023377,False,0.248169,10000,0,16
97,32,9,2776.536580,32.453638,0.023377,False,0.179735,10000,0,16
98,32,9,2776.536580,32.453638,0.023377,True,0.078808,10000,0,16
